In [4]:
import sys
import glob
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nc_path = '/Users/research/Documents/GitHub/gliderProcessing/glider_data/echo_20230113/delayed/nc/'
processing_mode = 'delayed'

traj_file_name = sorted(glob.glob(nc_path + '*{}*.nc'.format('trajectory')))
data = xr.open_dataset(traj_file_name[0], engine='netcdf4', decode_times=False)
#raw_data = xr.open_dataset(traj_file_name[0], engine='netcdf4', group='glider_record', decode_times=False)#
#raw_data = raw_data.sortby('time').to_dataframe().reset_index()
#data = data.sortby('time').to_dataframe().reset_index()



In [14]:
deployment_name = data.deployment_name.split('T')[0]
print(deployment_name+'_trajectory_file')

echo-2023-01-13_trajectory


## Correct lon/lat dead reckoning

In [ ]:
# Path containing python functions
path = ['/Users/research/Documents/GitHub/gliderProcessing/glider_processing_scripts/']
sys.path.append(path[0])

from gliderfuncs import correct_dead_reckoning, findProfiles

data['lon_qc'],data['lat_qc'] = correct_dead_reckoning(raw_data['m_lon'],raw_data['m_lat'],raw_data['time'],raw_data['x_dr_state'],raw_data['m_gps_lon'],raw_data['m_gps_lat'])


## Find Profiles

In [ ]:
data['profile_index'],data['profile_direction']=findProfiles(data['time'],data['depth'],stall=20,shake=200)

## Data QC

In [ ]:
from quartod_qc import quartod_qc_checks

In [ ]:
# set the variable we want to check and test 


qc_options = {
    'sensor_min': -2,
    'sensor_max': 40,
    'sensor_user_min': -1.8,
    'sensor_user_max': 40,
    'spike_thrshld_low': 0.5,
    'spike_thrshld_high': 1.5,
    'n_dev': 1,          # Adjust this value to represent the max acceptable rate of change
    'time_dev': 60/3600, # Set to the upper bound of the time range you're interested in (60 seconds)
    'min_wind_size': 100,
    'eps': 1e-6,
    'rep_cnt_fail': 3,
    'rep_cnt_suspect': 3
}

var = data['oxygen_concentration']
var_qc_flag = quartod_qc_checks(var.values, data['time'].values,'oxygen')

var_qc = var.copy()
var_qc[(var_qc_flag==3) | (var_qc_flag==4) | (var_qc_flag==9)] = np.nan


# Create the figure and subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
ax1.plot(np.array(data['time'], dtype='datetime64'), var,'.',color='blue')
ax2.plot(np.array(data['time'], dtype='datetime64'), var_qc, '.',color='red')
ax2.set_xlim(ax1.get_xlim())
fig.tight_layout()
plt.show()

In [ ]:
'''
from data2attr import save_netcdf
fname='echo-trajectory_file.nc'
gliders_db = '/Users/research/Documents/GitHub/gliderProcessing/glider_reference_information/glider_serial-numbers_and_sensor-serial-numbers.csv'
metadata_source = '/Users/research/Documents/GitHub/gliderProcessing/glider_data/echo_20230113/metadata.yml'
encoder = '/Users/research/Documents/GitHub/gliderProcessing/glider_processing_scripts/attributes/glider_dac_3.0_conventions.yml'

source_info = {
    'gliders_db': gliders_db,
    'metadata_source': metadata_source,
    'encoder': encoder,
    'processing_mode': 'delayed',
    'data_type': 'trajectory',
    'data_source': files,
    'filename': fname,
    'filepath': nc_path
}

save_netcdf(data, raw_data,source_info)



## PLOTS

In [ ]:
'''

fig, ax = plt.subplots(figsize=(14,7))
plt.plot(data['time'],data['depth'])
plt.gca().invert_yaxis()
ax.set(xlabel='time', ylabel='depth(m)',
       title='Glider Dives')


fig, ax = plt.subplots(figsize=(14,7))
idx_down = np.flatnonzero(data['profile_index'] == 1)
idx_up = np.flatnonzero(data['profile_index'] == 2)
ax.plot(data['time'].values[idx_down], data['depth'].values[idx_down], 'o', label='down cast')
ax.plot(data['time'].values[idx_up], data['depth'].values[idx_up], 'o', label='up cast')
ax.set(xlabel='time (s)', ylabel='depth (m)',
       title='Algorithm for Profile Splitting Code')
plt.gca().invert_yaxis()
ax.grid()
ax.legend()
plt.show()


fig, ax = plt.subplots(figsize=(14,7))
ax.plot(data['lon_qc'],data['lat_qc'],'o',label='corrected(underwater)')
ax.plot(raw_data['m_gps_lon'],raw_data['m_gps_lat'],'o',label='GPS(surface)')
ax.set(xlabel='longitude', ylabel='latitude',
       title='Algorithm for Deadreckoning Position Correction')
ax.grid()
ax.legend()
plt.show()


'''

